In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, stride=2, padding=1)
        self.fc1 = nn.Linear(32*14*14, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, 32*14*14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [3]:
# create data transform, converts from PIL image to tensor
# can add normalization options here
transform = transforms.Compose([transforms.ToTensor()])

# load the data
train_data = datasets.MNIST('data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('data', train=False, download=True, transform=transform)

# create a "dataloader" to generate batches
batch_size = 500
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [4]:
# instantiate the model
model = MNISTModel()
opt = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model.cuda()   # move the model to the gpu
model.train()  # prepares the model for training

epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()

        output = model(x)
        loss = criterion(output, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        y_pred = torch.argmax(output, -1)
        
        epoch_loss += loss
        epoch_correct += torch.eq(y, y_pred).float().sum()
        epoch_count += len(x)
    
    epoch_acc = epoch_correct / epoch_count
        
    print(f'epoch {epoch}: loss={epoch_loss:.5f}, accuracy={epoch_acc:.5f}')


epoch 0: loss=75.75027, accuracy=0.80605
epoch 1: loss=20.93831, accuracy=0.94772
epoch 2: loss=12.68884, accuracy=0.96815
epoch 3: loss=9.13002, accuracy=0.97718
epoch 4: loss=6.63744, accuracy=0.98363
epoch 5: loss=5.17621, accuracy=0.98710
epoch 6: loss=4.27787, accuracy=0.98920
epoch 7: loss=3.31874, accuracy=0.99167
epoch 8: loss=2.41796, accuracy=0.99423
epoch 9: loss=1.78591, accuracy=0.99580


In [6]:
# evaluate
test_correct = 0
test_count = 0

for x, y in test_loader:
    x = x.cuda()
    y = y.cuda()
    
    output = model(x)
    loss = criterion(output, y)
    y_pred = torch.argmax(output, -1)
    
    test_correct += torch.eq(y, y_pred).float().sum()
    test_count += len(x)

test_acc = test_correct / test_count
print(f'test accuracy={test_acc:.5f}')

test accuracy=0.98340
